# Mehrstufige Extraktion von Metadateninformationen aus sächsischen OER Inhalten

Sebastian Zug, André Dietrich 
TU Bergakademie Freiberg, Institut für Informatik

In [ ]:
import pandas as pd
from pathlib import Path
import difflib

## 1. Schritt: Bewertung der Metadaten von OER Inhalten aus dem LMS OPAL

In [ ]:
data_folder =  "/mnt/9cd5c6a1-07f3-4580-be34-8d8dd9d6fe6d/Connected_Lecturers/Opal/raw/"
data_file_attribs = "OPAL_files_attrib.p"
data_file_meta = "OPAL_files_meta.p"

In [ ]:
df_files = pd.read_pickle(Path(data_folder) / data_file_attribs)

In [ ]:
def generateEmptinessStatistics(df):
    df_empty = df.apply(lambda x: x == '').sum(axis=0).rename("count").to_frame()
    df_empty['empty_in_percent'] = df_empty / len(df)
    return df_empty 

In [ ]:
generateEmptinessStatistics(df_files)

Ok, wer sind die metainformationsspezifischen Vorbilder?

In [ ]:
df_files[df_files["opal:creator"]!=""]['opal:creator'].value_counts().head(10)

## Schritt 2: Traditionelle Aggregation der Metadaten

Welche Dateitypen sind denn  in den OPAL OER Daten überhaupt vertreten?

In [ ]:
df_files["pipe:file_type"].value_counts().head(10)

Im Rahmen der ersten Untersuchungsreihe fokussieren wir uns auf {`pdf`, `docx`, `pptx`, `xlsx`}. Ausgangspunkt war eine Stichprobenprüfung der Mediendateien, die keinerlei Metadaten enthielten.

In [ ]:
relevant_file_type = ['pdf', 'docx', 'pptx', 'xlsx']
df_files[df_files["pipe:file_type"].isin(relevant_file_type)]["pipe:file_type"].shape[0] / df_files.shape[0]

Offenbar decken wir mit dieser Auswahl etwa 55 Prozent der in OPAL enthaltenen Dokumente ab. Für diese Datentypen existieren unterschiedliche Metadateninformationen, die in die Dateien eingebettet sind. 

![](https://redmondmag.com/articles/2019/02/25/~/media/ECG/redmondmag/Images/2019/02/0225red_metadata1.ashx)

Ein Skript der CL Pipeline hat diese, sofern vorhanden, für uns ausgelesen ...

In [ ]:
df_meta = pd.read_pickle(Path(data_folder) / data_file_meta)
generateEmptinessStatistics(df_meta)

Für 60 Prozent der Autoren können wir zumindest einen Vorschlage zum Namen machen? Das klingt schon mal vielversprechend. Lassen Sie uns die Metadaten der verschiedenen Quellen vergleichen, um die Qualitäten zu prüfen.

In [ ]:
# merge df_files and df_meta on "opal:file_id"
df_merge = pd.merge(df_meta, df_files.drop(['pipe:file_type'], axis=1), on="pipe:ID", how="left")

Schauen wir uns zunächst positiv Beispiel an, die aus beiden Quellen enthielten.

In [ ]:
df_merge[(df_merge["opal:creator"]!="") &  (df_merge["file:author"]!="")][["opal:creator", "file:author"]].head(15)

Und nun umgekehrt, helfen uns die aus den Dateien extrahierten Metadaten bei der Identifikation der Autoren? Das Bild ist durchwachsen ...

In [ ]:
df_merge[(df_merge["opal:creator"]=="") &  (df_merge["file:author"]!="")][["opal:creator", "file:author"]].head(15)

Wir brauchen einen weiteren Mechanismus der uns bei der Extraktion unterstützt! 

### Schritt 3: KI basierte Metadatenaggregation

Die CL Pipeline wurde um ein weiteres Modul erweitert, dass aktuell ein llama3 LLM verwendet, um die Titel, Autoren und Keyworte zu extrahieren.

In [ ]:
data_file_aimeta = "OPAL_ai_meta.p"
df_aimeta = pd.read_pickle(Path(data_folder) / data_file_aimeta)

In [ ]:
df_aimeta.shape[0] / df_files[df_files["pipe:file_type"].isin(relevant_file_type)]["pipe:file_type"].shape[0] 

Für etwa 90 Prozent der Office-Dateien und pdfs konnten entsprechende Informationen erschlossen werden. Die Frage ist, wie schlüssig diese Resultate sind. Fusionieren wir also alle Datensätze miteinander, um das zu untersuchen.

In [ ]:
df_merge_all = pd.merge(df_aimeta, df_merge.drop(['pipe:file_type'], axis=1), on="pipe:ID", how="left")
df_merge_all.iloc[0]

Schauen wir uns also zunächst die Datensätze an, die sowohl Metadaten aus OPAL mitbrachten, für die aber auch Daten generiert werden konnten.

In [ ]:
df_merge_all[ (df_merge_all["opal:creator"]!="") &
             ((df_merge_all["file:author"]!="") | (df_merge_all["ai:author"]!=""))][["opal:creator", "file:author", "ai:author", "pipe:file_type"]].tail(20)

... und umgekehrt? Wo können `ai` und `file` zum Beispiel Lücken bei den Autoren füllen? 

In [ ]:
df_merge_all = df_merge_all[~df_merge_all['ai:author'].str.contains("Ich kann")]

In [ ]:
df_merge_all[ (df_merge_all["opal:creator"]=="") &
             ((df_merge_all["file:author"]!="") | (df_merge_all["ai:author"]!=""))][["file:author", "ai:author", "pipe:file_type"]].tail(20)

In [ ]:
A =df_merge_all[
             ((df_merge_all["file:author"]!="") & (df_merge_all["ai:author"]!=""))][["file:author", "ai:author", "pipe:file_type"]]

In [ ]:
def check_substrings(row):
    if row["file:author"]!="" and row["ai:author"]!="" and row["file:author"]!=None and row["ai:author"]!=None:
        input_a = row["file:author"]
        input_b = row["ai:author"]
        s = difflib.SequenceMatcher(None, input_a, input_b)
        return s.find_longest_match(0, len(input_a), 0, len(input_b)).size
    else:
        return 0

A['common_substring_count'] = A.apply(check_substrings, axis=1)

In [ ]:
A[A.common_substring_count > 5].shape